As the data was originally stored in an RData file, we will need to read and make some preprocessing before passing it on to Julia. To do this we will use the RCall.jl package. Note that using it requires that R is installed in the users computer. In particular, we will use the new\_168\_2024\_HORARIOS\_pm2\_n\_R.RData file which can be converted into an $M \times 173$, $M = 761$ being the total numer of curves, with the following columns: 
1. Station (String), corresponds to the station of a curve (row). 
2. unit\_n, this value equals the week number of a curve minus one. i.e. week number of a particular row = unit\_n + 1. 
3. date
4. date\_lower
5. date\_upper
6. Hourt, $t \in \{1,2,...,168\}$. 

In [1]:
using RCall, LinearAlgebra


R""" 
#We will load the dplyr package for easier data management in R. 
#Note that, most likely, this will print a warning fromo RCall.jl. This is normal. 
library(dplyr) #Loads the dplyr package into R. 

 data <- load("new_168_2024_HORARIOS_pm2_n_R.RData") #Loads the desired file into R's workspace.
 data <- r_data #Obtains the data frame with the desired data. 
 head(data) #Prints the top rows of data. 

 #For confort, we will now convert unit_n to be the actuall week number of the curves.

 data <- data%>%mutate(unit_n = unit_n + 1) #Sets unit_n = unit_n + 1. 
 Y_obs <- as.matrix(data[, 6:173])# Gets an M x T, T = 168, matrix with the functional observations. 
                                  #Note that these are already ordered by Station (i.e. by their alphabetical order), 
                                  #and unit_n in an increasing fashion. 
 head(Y_obs) #Prints the top rows of the curves. 

 #We may observe that we already have one of the inputs for flfosr, we are only missing a design matrix and an M_rep vector. 
 #Before obtaning this components, we will give some explanation of what we are going to do. 
 """

┌ Warning: RCall.jl: 
│ Adjuntando el paquete: 'dplyr'
│ 
│ The following objects are masked from 'package:stats':
│ 
│     filter, lag
│ 
│ The following objects are masked from 'package:base':
│ 
│     intersect, setdiff, setequal, union
│ 
└ @ RCall C:\Users\End User\.julia\packages\RCall\WTNSB\src\io.jl:171


RObject{RealSxp}
  Hour1 Hour2 Hour3 Hour4 Hour5 Hour6 Hour7 Hour8 Hour9 Hour10 Hour11 Hour12
0    10     6     3     4     4    14     5     2     4      6     15     17
1    10   NaN    13   NaN    13   NaN     8   NaN     5    NaN      3    NaN
2     8    16    17    16    21    19    16    12     8     10     15     22
3    22    22    18    19    20    22    24    27    38     43     45     43
4     6   NaN     6   NaN     3   NaN     8   NaN     4    NaN    NaN    NaN
5     3     3     2     6     2     1   NaN     0     1      4      5      3
  Hour13 Hour14 Hour15 Hour16 Hour17 Hour18 Hour19 Hour20 Hour21 Hour22 Hour23
0     12     20     22     26     27      6    NaN      3      4      9     11
1      2    NaN      1    NaN      3    NaN      2    NaN      3    NaN     12
2     18     20     12     17     22     28     18     19      7      5      3
3     44     55     41     32     21     25     32     33     24     24     16
4      1    NaN    NaN    NaN      3    NaN      

Let us recall that the FMM presented in Johansson L.(2025), with a mix of our and its notation, is of the form 

$$Y_{ij}(\tau) = \mu(\tau) + \eta_j(\tau) +  \gamma_i(\tau) + \omega_{ij}(\tau) + \varepsilon_{ij}(\tau)$$

where $\mu(\tau)$ represents a global mean, $\eta_j(\tau)$ corresponds to a global week level fixed effect, $\gamma_i(\tau)$ is a subject specific random effect that in the abscence of $\eta_j(\tau)$ determines between subject variation, and $\omega_{ij}(\tau)$ as a subject-visit specific efect which deteremines within subject variation. 

How should this translate into our model and our design matrix. For starters, for identifiability,  so our design matrix should include a column with all ones. i.e. the first one. 


To make this 

In [ ]:


R""" 
library(pracma) #Loads the pracma package, this contains the function ones which will be used 
                #later. 

 M <- dim(Y_obs)[1] #Gets the total number of curves M = M1+M2+...+MN
 T <- dim(Y_obs)[2] #Gets the total number of hour recordings. 

 unique_unit_n <- sort(unique(data$unit_n)) #Gets the unique recorded weeks. 
 L <- length(unique_unit_n) #Gets L, in this case the total number of unique weeks. 


 X <- ones(M, L) #Creates an M x L matrix filled with ones. 
                   #X will be modified to obtain our final design matrix. 

unit_n <- data$unit_n #Gets the column with the week number. 

#The following for modifies the columns of X (except for the first one)
#and sets it to a column of zeros and ones where a curve has a one entry if
#it was observed on the week unique_unit_n[l]. 

#The previous is accomplished by taking unit_n, and comparing it entry by entry to 
#to a vector filled with the value unique_unit_m[l]. i.e. unit_n == rep(unique_unit_n[l], M)
#produces an M-dimensional vector filled with TRUE and FALSE values and
#, when implementing as.numeric, it becomes a vector of 0's and 1's. 

for (l in 1:(L))
    X[, l] <- as.numeric(unit_n == rep(unique_unit_n[l], M) )

# head(X) #Prints the top rows of the design matrix X. 
Station <- as.numeric(table(data$Station))



 """


AJM AJU BJU CAM CCA FAR GAM HGM INN MER MON MPA NEZ PED SAC SAG TLA UAX UIZ 
 48  40  43  47  41  46  35  30  37  48  27  40  37  49  46  43  36  32  36 


RObject{RealSxp}
 [1] 48 40 43 47 41 46 35 30 37 48 27 40 37 49 46 43 36 32 36


Given this, we now have all of the ingredients to run FastBayesFMMs! We only need to pass everything to Julia. Fortunately, as Station, Y\_obs, and X are vector and matrices respectively. Thus, we only need to pass them to Julia. 